In [4]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});


<IPython.core.display.Javascript object>

## Section 1: Model complexity for non-equilibirum dynamics

Here I sketch the derivation of Model Complexity for a non-equilibrium Langevin dynamics. This derivation is based on the results of Haas et. al., "Analysis of Trajectory Entropy for Continuous Stochastic Processes at Equilibrium" (2014).


Our goal is to calculate KL divergence between the trajectories generated from a distribution of interest $\mathcal{P}[X(t)]$ and trajectories generated from the reference distribution $\mathcal{Q}[X(t)]$ (almost the same as Eq. (1) in Haas paper):
\begin{equation}\label{eq1}
S\equiv-\int_0^{\infty}\mathcal{D}X(t)\mathcal{P}[X(t)]\ln\frac{\mathcal{P}[X(t)]}{\mathcal{P}[X(t)]}.
\end{equation}
Here we take the integral until infinity, but in numerical calculations we just need to integrate until sufficiently large time so that nearly all of the trajectories are absorbed, and both probabilities are close to zero. Following Haas, we discretize the path into time intervals $\Delta t$ and take the continuum limit $\Delta t \to 0$ (same as Eq.(3) in Haas paper):
\begin{equation}\label{eq2}
    S=\lim_{\Delta t \to 0} -\int dx_0 \int dx_1 \cdots \int dx_N P(X)\ln \frac{P(X)}{Q(X)}.
\end{equation}
Following Haas paper, we use Markov factorization $P(X)=p(x_0)\prod_{\tau=0}^N p(x_{\tau+1}|x_\tau)$, and the same thing for $Q(X)$. As a result, we obtain (same as Eq. (4) in Haas paper, and also dropping the limit for clarity/conciseness, and also correcting Haas's typo highlighted in red):
\begin{equation}\label{eq3}
S=-\int\left(\prod_{\tau'=0}^Ndx_{\tau'}\color{red}{dx_{N+1}}p(x_0)p(x_{\tau'+1}|x_{\tau'})\right)\left(\ln\frac{p(x_0)}{q(x_0)}+\sum_{\tau=0}^N\ln\frac{p(x_{\tau+1}|x_{\tau})}{q(x_{\tau+1}|x_{\tau})}\right).
\end{equation}
This integral can be reqorganized (similar to Eq.(5) in Haas, but he has a typo and there is a term missing. The missing term is highlighted in red):

\begin{eqnarray}\label{eq4}
S&=-\sum_{\tau=0}^N\int dx_{\tau+1}dx_\tau p(x_{\tau+1}|x_\tau) \ln\frac{p(x_{\tau+1}|x_{\tau})}{q(x_{\tau+1}|x_{\tau})} \left(\int \prod_{\substack{\tau'\neq\tau \\ \tau'\neq \tau+1}}^Ndx_{\tau'}dx_{N+1}p(x_0)p(x_{\tau'+1}|x_{\tau'})\color{red}{p(x_{\tau+2}|x_{\tau+1})}\right)  \\ \nonumber
&- \int dx_0\left(\int\prod_{\tau'=1}^Ndx_{\tau'}dx_{N+1}p(x_0)p(x_{\tau'+1}|x_{\tau'})\right)\ln\frac{p(x_0)}{q(x_0)}
\end{eqnarray}

We can simplify some of the terms in Eq. (\ref{eq4}):
\begin{eqnarray}\label{eq5}
&\int \prod_{\substack{\tau'\neq\tau \\ \tau'\neq \tau+1}}^Ndx_{\tau'}dx_{N+1}p(x_0)p(x_{\tau'+1}|x_{\tau'})p(x_{\tau+2}|x_{\tau+1})= \\ \nonumber
&= \left(\int dx_{\tau+2} p(x_{\tau+2}|x_{\tau+1})\cdots\left(\int dx_N p(x_{N}|x_{N-1})\left(\int dx_{N+1}p(x_{N+1}|x_{N})\right)\right)\cdots\right)\times \\ \nonumber
&\times \left(\int dx_{\tau-1} p(x_{\tau}|x_{\tau-1})\cdots\left(\int dx_1 p(x_{2}|x_{1})\left(\int dx_{0}p(x_{1}|x_{0})p(x_0)\right)\right)\cdots\right) = p(x_{\tau}).
\end{eqnarray}

\begin{eqnarray}\label{eq6}
&\int\prod_{\tau'=1}^Ndx_{\tau'}dx_{N+1}p(x_0)p(x_{\tau'+1}|x_{\tau'})= \\ \nonumber
&=p(x_0)\left(\int dx_{1} p(x_2|x_1)\cdots\left(\int dx_N p(x_{N}|x_{N-1})\left(\int dx_{N+1}p(x_{N+1}|x_{N})\right)\right)\cdots\right)=p(x_0)
\end{eqnarray}

Substituting Eq. (\ref{eq5}) and Eq. (\ref{eq6}) into Eq. (\ref{eq4}), we obtain (cf. Eq. (6) in the paper):
\begin{equation}\label{eq7}
S = -\sum_{\tau=0}^N \int dx_\tau dx_{\tau+1} p(x_{\tau+1},x_\tau) \ln\frac{p(x_{\tau+1}|x_{\tau})} {q(x_{\tau+1}|x_{\tau})} - \int dx_0 p(x_0) \ln \frac{p(x_0)}{q(x_0)}
\end{equation}
Since we are working with non-equilibrium dynamics, the terms under the sum are not identical (as the probability distribution is evolving). Therefore, we cannot replace sum with $t_{obs}/\Delta t$ coefficient, but we have to take a sum over the time-varying distributions $p$ and $q$. Also, at $t=0$ these distributions are given initial distributions that are different from the equilibirum distributions.

Following Haas, we denote the second term in Eq. (\ref{eq7}) as $S_{\rm eq}$. This term can be calculated numerically in a streight-forward fashion. For reference distribution we may choose some wide unspecific distribution that is zero on the boundaries, such as cosyne squared.

Following Haas, we rewrite Eq. (\ref{eq7}) through the following terms:
\begin{equation}\label{eq8}
S = S_{\rm eq}+\lim_{\Delta t \to 0}\sum_{\Delta t} S_{\rm KL}(\Delta t,\tau) = S_{eq} +\int_{0}^{\infty}d\tau \lim_{\Delta t \to 0} \frac{S_{\rm KL}(\Delta t, \tau)}{\Delta t} ,
\end{equation}
where:
\begin{equation}\label{eq9}
S_{\rm KL} (\Delta t, \tau) = -\int dx_\tau dx_{\tau+\Delta t} p(x_{\tau+\Delta t},x_\tau) \ln\frac{p(x_{\tau+\Delta t}|x_{\tau})} {q(x_{\tau+\Delta t}|x_{\tau})}
\end{equation}

Following Haas, we replcace the limit with time derivative and $t=\tau$:
\begin{eqnarray}\label{eq10}
&\lim_{\Delta t \to 0} \frac{S_{\rm KL}(\Delta t,\tau)}{\Delta t} = -\frac{\partial\left(\int dx_\tau dx_{\tau+ t} p(x_{\tau+t},x_\tau) \ln\frac{p(x_{\tau+t}|x_{\tau})} {q(x_{\tau+t}|x_{\tau})}\right)}{\partial t}\Biggr|_{ t=0} = \\
&= -\mathop{\mathbb{E}}_{p(x_\tau)}\left[\frac{\partial}{\partial t}\left(\int dx_{t+\tau}p(x_{\tau+t}|x_\tau) \ln\frac{p(x_{\tau+t}|x_{\tau)}}{q(x_{\tau+t}|x_{\tau})}\right)\Biggr|_{ t=0}\right]
\end{eqnarray}

Following Haas, we interchange time derivative with an integral:, 
\begin{eqnarray}\label{eq11}
&\frac{\partial}{\partial t}\left(\int dx_{t+\tau}p(x_{\tau+t}|x_\tau) \ln\frac{p(x_{\tau+t}|x_{\tau)}}{q(x_{\tau+t}|x_{\tau})}\right) = \\
&= \int dx_{t+\tau}\left[\frac{\partial}{\partial t}\left(p(x_{\tau+t}|x_\tau) \ln\frac{p(x_{\tau+t}|x_{\tau)}}{q(x_{\tau+t}|x_{\tau})}\right)\Biggr|_{t=0}\right] 
\end{eqnarray}
The time derivatives are:
\begin{eqnarray}\label{eq12}
&\frac{\partial}{\partial t}\left(p(x_{\tau+t}|x_\tau) \ln\frac{p(x_{\tau+t}|x_{\tau)}}{q(x_{\tau+t}|x_{\tau})}\right)\Biggr|_{t=0} = \frac{\partial p(x_{\tau+t}|x_\tau) }{\partial t}\ln\frac{p(x_{\tau+t}|x_{\tau)}}{q(x_{\tau+t}|x_{\tau})} + \\
&+\frac{1}{q(x_{\tau+t}|x_\tau)} \left(\frac{\partial p(x_{\tau+t}|x_\tau) }{\partial t}q(x_{\tau+t}|x_\tau) - \frac{\partial q(x_{\tau+t}|x_\tau) }{\partial t}p(x_{\tau+t}|x_\tau)\right)  \Biggr|_{t=0} = \\
&=\left(\frac{\partial p(x_{\tau+t}|x_\tau) }{\partial t} - \frac{\partial q(x_{\tau+t}|x_\tau) }{\partial t}\right)  \Biggr|_{t=0},
\end{eqnarray}
where in the transition we used that for $t=0$, $p(x_{\tau+t}|x_{\tau})=q(x_{\tau+t}|x_{\tau})=\delta(x_{\tau+t}-x_{\tau})$.

Eqs. (\ref{eq10}), (\ref{eq11}), (\ref{eq12}) bring us to the following result (cf. Eq. (12) in the paper):
\begin{equation}\label{eq13}
\lim_{\Delta t \to 0} \frac{S_{\rm KL}(\Delta t,\tau)}{\Delta t} =-\mathop{\mathbb{E}}_{p(x_\tau)} \left[\int dx_{t+\tau} \left(\frac{\partial p(x_{\tau+t}|x_\tau) }{\partial t} - \frac{\partial q(x_{\tau+t}|x_\tau) }{\partial t}\right)  \Biggr|_{t=0}\right]
\end{equation}

Next we use the Fokker-Planck equation for time-dependent probability distributions:
\begin{equation}\label{eq14}
\frac{\partial p(x_t|x_0)}{\partial t} = \frac{\partial}{\partial x_t} \left(D\frac{\partial p(x_t|x_0)}{\partial x_t}-DF(x_t)p(x_t|x_0)\right)
\end{equation}

However, instead of Eq. (\ref{eq14}), Haas suggests to use a scaled version ($\rho(x_t,x_0)=p(x_t|x_0) p_{\rm eq}(x_0)/\sqrt{p_{\rm eq}x_t}$, note a error in the paper where peq(x0) is under the square root ). The FP equation for the rescaled version reads:
\begin{equation}\label{eq15}
\frac{\partial \rho(x_t,x_0)}{\partial t} = D\frac{\partial^2 \rho(x_t,x_0)}{\partial x_t^2} - \left(\frac{F^2(x_t)D}{4}+\frac{F'(x_t)D}{2}\right)\rho(x_t,x_0).
\end{equation}

It is not clear why Eq. (\ref{eq15}) can be substitued into Eq. (\ref{eq13}) directly, since the normalizations $(p(x_t))^{1/2}$ are different for $p$ and $q$ distribution. Therefore, the coefficients in front of the two terms in Eq. (\ref{eq13}) will be different, and the second derivatives will not cancel out as Haas suggests. It is not clear how to do a correct derivation. However, assuming that this step can be made, we substitute Eq. (\ref{eq15}) in to Eq. (\ref{eq13}) and assuming $p$ and $q$ have the same diffusion, and also that $q$ has zero Force, we are left with the follwing equation (where the limit $t\to 0$ is evaluated by substituting a delta function):
\begin{equation}\label{eq16}
\lim_{\Delta t \to 0} \frac{S_{\rm KL}(\Delta t,\tau)}{\Delta t} =\mathop{\mathbb{E}}_{p(x_\tau)} \left[\int dx_t \left(\frac{F^2(x_t)D}{4}+\frac{F'(x_t)D}{2}\right)\delta(x_t-x_0)\right].
\end{equation}
In Eq. (\ref{eq16}) we can evaluate the integral over $x_t$ to obtain:
\begin{equation}\label{eq17}
\lim_{\Delta t \to 0} \frac{S_{\rm KL}(\Delta t,\tau)}{\Delta t} =\int dx_0 p_\tau (x_0)\left(\frac{F^2(x_0)D}{4}+\frac{F'(x_0)D}{2}\right).
\end{equation}
Note that in Eq. (\ref{eq17}) the distribution $p_\tau (x)$ dependes on the time $\tau$ and is not equal to the equlibrium distribution. Thus, this equation is NOT equvivalent to the following result obtained in the paper:
\begin{equation}\label{eq18}
\lim_{\Delta t \to 0} \frac{S_{\rm KL}(\Delta t,\tau)}{\Delta t} =-\frac{D}{4}\int dx_0 p_\tau (x_0)F^2(x_0).
\end{equation}

The full entropy can be obtained by Eq. (\ref{eq8}):
\begin{equation}\label{eq19}
S = S_{eq} +\int_{0}^{\infty}d\tau \lim_{\Delta t \to 0} \frac{S_{\rm KL}(\Delta t,\tau)}{\Delta t} ,
\end{equation}
Where the first term is KL-divergence between the initial distributions $p_0(x)$ and $q_0(x)$, and the second term was evaluated in Haas paper (but we are not sure whether the derivation is correct) by Eq. (\ref{eq17}) or (\ref{eq18}) which could not hold for our non-equlibrium case.



## Section 2: Numerical simulations

### Evaluating space-time integrals
Eq. (22),(23) suggest we need to be able to evaluate space-time integrals (STI) like the following one:
\begin{equation}
{\rm STI} = \int_0^\infty dt\int_{-1}^1 dx p(x,t)F^2(x)
\end{equation}

These expressions can be evaluated by using the symmetric FPE Eq. (19) for the scaled probability density, where the r.h.s. is represented by the operator $-\boldsymbol{\mathcal{H}}$ and where we switch to the eigenbasis of this operator: 
\begin{eqnarray}\label{eq1}
{\rm STI} = \int_0^\infty dt\int_{-1}^1 dx p(x,t)F^2(x) = \int_0^\infty dt\int_{-1}^1 dx \rho_0(x)e^{-\boldsymbol{\mathcal{H}}t}F^2(x)\sqrt{p_{\rm eq}(x)} = \\
= \int_0^\infty dt \int_{-1}^1 dx \sum_k \Psi_k(x) \rho_{0,k} e^{-\lambda_k t}\sum_l \Psi_l(x)(F^2\sqrt{p_{\rm eq}})_l = \sum_k \frac{\rho_{0,k} (F^2\sqrt{p_{\rm eq}})_k}{\lambda_k},
\end{eqnarray}
where we took the time integral analytically, and also used the orthnormal properties of the egenfunctions $\Psi_k(x)$.

Thus, to calculate space-time integral of any function $g(x)$, we multiply it by $\sqrt{p_{\rm eq}(x)}$, transform it into the dark basis, we also transform into the dark basis scaled initial probability distribution $\rho_{0}(x)=p_0(x)/\sqrt{p_{eq}(x)}$, and then sum up the product of these two terms divided by the eigenvalues.

We can check that Eq. (\ref{eq1}) works in practice by generating a trajectory from the some dynamics, and then computing the function (such as $F^2(x)$) along these trajectories: $<F^2(x)>_{X(t)}=(1/t_{\rm trial})\sum_l F^2(x_l)$, where the summation is carried out along all of the generated trajectories, and $t_{\rm trial}$ is an average trial duration. This should give us the same results, as Eq. (\ref{eq1}).

In [1]:
# 1) git clone brain_flow package
# 2) git checkout public
# 3) go to brain_flow/energy_model and run compile.sh
# 4) Add to path

import sys
#sys.path.append('/Users/mikhailgenkin/temp/BrainFlow')

In [2]:
import neuralflow

In [3]:
import os, json, numpy as np
# from FiringRatesModeling.MyDBClass import myDB

#trying to solve the c_get_gamma.pyx 
# from setuptools import setup
# from Cython.Build import cythonize
# sys.path.append('BrainFlow/brain_flow/energy_model')
# setup(ext_modules=cythonize('c_get_gamma.pyx'))

from neuralflow import energy_model

em_gt = energy_model.EnergyModel(peq_model={"model": "linear_pot",
                                           "params": {"slope": -2.65}}, 
                                 pde_solve = energy_model.PDESolve(Np=8,Ne=32),
                                 
                                p0_model={"model":"cos_square","params": {}},
                                )

lQ, _, Qx = em_gt.pde_solve_.solve_EV(em_gt.peq_[0], em_gt.D_[0], q=None, w=em_gt.peq_[0], mode='h0', fr=None, Nv=em_gt.Nv)
Force=em_gt.dmat_d.dot(np.log(em_gt.peq_[0]))

In [4]:
#First, let us evaluate the second term in Eq. (8) by using Eq. (9)
deltaT = 0.001
t_terminal = 10
#Calculate transition probability for the model of interest p(x_{tau+delta t}|x_{tau}) 
# by translating e^-lambda*t into SEM basis
p = Qx.dot(np.diag(np.exp(-lQ*deltaT))).dot(Qx.T).dot(np.diag(em_gt.w_d))
#Need also to rescale it back by sqrt(peq(xt)) and sqrt(peq(x0))
p = np.sqrt(em_gt.peq_[0,:,np.newaxis])*p/np.sqrt(em_gt.peq_[0])
#Also get rid of embedded weights
p=p/em_gt.w_d
#For numerical stability of KL divergence
p[p<10**-10]=10**-10

#Calculate the same thing for the reference model with constant peq
qeq = 0.5*np.ones_like(em_gt.x_d)
lQ2, QxOrig2, Qx2 = em_gt.pde_solve_.solve_EV(qeq, em_gt.D, q=None, w=qeq, mode='h0', fr=None, Nv=em_gt.Nv)
q = Qx2.dot(np.diag(np.exp(-lQ2*deltaT))).dot(Qx2.T).dot(np.diag(em_gt.w_d))
q = np.sqrt(qeq[:,np.newaxis])*q/np.sqrt(qeq)
q=q/em_gt.w_d
q[q<10**-10]=10**-10

#Now use Eq. (9) to calculate the second term of Eq. (8). 
#Here time integral and both of the spatial integral are evaluated numerically

S2_Eq9 = 0
ptau=em_gt.p0_[0]
for time in np.arange(0,t_terminal,deltaT):
    KL=p*ptau*np.log(p/q) #KL divergence
    S2_Eq9+=np.sum(np.sum(KL*em_gt.w_d,axis=1)*em_gt.w_d) #update time integral
    ptau=np.sum(p*ptau*em_gt.w_d,axis=1) #update p_tau

#The result is S2_Eq9 = 2.928
    
#Now let us compare this result with the second term in Eq. (8) evaluated by Eq.(22):
Fs=em_gt.D_/4*Force**2
Fd = Qx.T.dot(np.diag(em_gt.w_d)).dot(Fs*np.sqrt(em_gt.peq_[0]))
rho0d= Qx.T.dot(np.diag(em_gt.w_d)).dot(em_gt.p0_[0]/np.sqrt(em_gt.peq_[0]))
S2_Eq22=np.sum(Fd*rho0d/lQ) #The result is 2.947

#Now let us compare this result with the second term in Eq. (8) evaluated by Eq.(21):
Fs=-(em_gt.D_/4*Force**2+em_gt.D_/2*em_gt.dmat_d.dot(Force))
Fd = Qx.T.dot(np.diag(em_gt.w_d)).dot(Fs*np.sqrt(em_gt.peq_[0]))
rho0d= Qx.T.dot(np.diag(em_gt.w_d)).dot(em_gt.p0_[0]/np.sqrt(em_gt.peq_[0]))
S2_Eq21=np.sum(Fd*rho0d/lQ) #The result is 2.947

print('S2 term evaluated by Eq. 8,9: {}'.format(S2_Eq9))
print('S2 term evaluated by Eq. 8,21: {}'.format(S2_Eq21))
print('S2 term evaluated by Eq. 8,22: {}'.format(S2_Eq22))    


S2 term evaluated by Eq. 8,9: 9.624839108671852
S2 term evaluated by Eq. 8,21: -341037600207.63824
S2 term evaluated by Eq. 8,22: 341037600206.5334


In [5]:
p = Qx.dot(np.diag(np.exp(-lQ*deltaT))).dot(Qx.T).dot(np.diag(em_gt.w_d))

In [6]:
p = np.sqrt(em_gt.peq_[0,:,np.newaxis])*p/np.sqrt(em_gt.peq_[0])

In [7]:
em_gt.peq_[0,:,np.newaxis].shape

(225, 1)